In [ ]:
import os,sys
import h5py  
import numpy as np
import math,cmath
from matplotlib.backends.backend_pdf import PdfPages
from scipy.optimize import curve_fit, fsolve
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['figure.facecolor'] = 'white'

if not os.path.abspath('../util') in sys.path:
    sys.path.append(os.path.abspath('../util'))
import util as yu

ens='cA211.53.24'
lat=yu.LatticeEnsemble(ens)

In [ ]:
basePath='/p/project/pines/li47/code/projectData/nucleon_sigma_term/cA211.53.24/pion2pt/avg/P.h5'
with h5py.File(basePath) as f:
    cfgList=list(f.keys())
basePath='/p/project/pines/li47/code/projectData/nucleon_sigma_term/cA211.53.24/pion2pt/avg/pi0DoubleLoop.h5'
with h5py.File(basePath) as f:
    cfgList=list(set(cfgList)&set(f.keys()))

Ncfg=len(cfgList)
print(Ncfg)

data={}

basePath='/p/project/pines/li47/code/projectData/nucleon_sigma_term/cA211.53.24/pion2pt/avg/P.h5'
with h5py.File(basePath) as f:
    momList=f['0000/mvec'][()]
    for ky in ['P0DN','P0UP','PPDN','PPUP']:
        data[ky]=np.array([f[cfg][ky][:,:,0,0]+1j*f[cfg][ky][:,:,0,1] for cfg in cfgList])

basePath='/p/project/pines/li47/code/projectData/nucleon_sigma_term/cA211.53.24/pion2pt/avg/pi0DoubleLoop.h5'
with h5py.File(basePath) as f:
    for ky in ['pi0DoubleLoop']:
        data[ky]=np.array([f[cfg][ky] for cfg in cfgList])

basePath='/p/project/pines/li47/code/projectData/nucleon_sigma_term/cA211.53.24/pion2pt/avg/pi0Loop.h5'
with h5py.File(basePath) as f:
    for ky in ['pi0Loop_g5_vev']:
        data[ky]=np.array([f[cfg][ky][()] for cfg in cfgList])

momDic={}
for i in range(len(momList)):
    momDic[tuple(momList[i])]=i

t_total=48

In [ ]:
N2momListDic={}
ijkRange=range(-2,3)
Nmax=4
for n2 in range(Nmax+1):
    N2momListDic[n2]=[]
for i in ijkRange:
    for j in ijkRange:
        for k in ijkRange:
            n2=i**2+j**2+k**2
            if n2 > Nmax:
                continue
            N2momListDic[n2].append((i,j,k))

In [ ]:
def getDatByMom(case,mom):
    i_mom=momDic[mom]
    if case == 'piC':
        t=np.array([[(data['PPUP'][cfg,:,i_mom]+data['PPDN'][cfg,:,i_mom])/2,0] for cfg in range(len(cfgList))],dtype=object)
        return t
    elif case == 'pi0_conn':
        t=np.array([[(data['P0UP'][cfg,:,i_mom]+data['P0DN'][cfg,:,i_mom])/2,0] for cfg in range(len(cfgList))],dtype=object)
        return t
    elif case == 'pi0_disc':
        t=np.array([[data['pi0DoubleLoop'][cfg,:,i_mom],data['pi0Loop_g5_vev'][cfg]] for cfg in range(len(cfgList))],dtype=object)
        if i_mom!=momDic[(0,0,0)]:
            t=np.array([[data['pi0DoubleLoop'][cfg,:,i_mom],0] for cfg in range(len(cfgList))],dtype=object)
        return t
    elif case == 'pi0':
        t=np.array([[(data['P0UP'][cfg,:,i_mom]+data['P0DN'][cfg,:,i_mom])/2+data['pi0DoubleLoop'][cfg,:,i_mom],data['pi0Loop_g5_vev'][cfg]] for cfg in range(len(cfgList))],dtype=object)
        if i_mom!=momDic[(0,0,0)]:
            t=np.array([[(data['P0UP'][cfg,:,i_mom]+data['P0DN'][cfg,:,i_mom])/2+data['pi0DoubleLoop'][cfg,:,i_mom],0] for cfg in range(len(cfgList))],dtype=object)
        return t
    
def getDat(case):
    return np.mean([getDatByMom(case,mom) for mom in N2momListDic[n2]],axis=0)


(Nrow, Ncol) = (5, 2)
(Lrow, Lcol) = (4*2, 5*2)
fig, axs = plt.subplots(Nrow, Ncol, figsize=(Lcol*Ncol, Lrow*Nrow), squeeze=False)
yu.addRowHeader(axs,range(5))

for n2 in range(Nmax+1):
    irow=n2

    icol=0
    def tFunc(dat):
        t=np.mean(dat,axis=0)
        t=t[0]-(-t[1]**2)
        t=np.real(t)
        return t
    for case in ['piC','pi0_conn','pi0_disc','pi0']:
        dat=getDat(case)
        (mean,cov,err)=yu.jackknife(dat,tFunc)
        
        tmin=2;tmax=45+1
        tMean=mean[tmin:tmax];tErr=err[tmin:tmax]

        colorDic={'piC':'r','pi0':'g','pi0_conn':'b','pi0_disc':'purple'}
        axs[irow,icol].errorbar(range(tmin,tmax),tMean,tErr,color=colorDic[case],fmt='s',elinewidth=2,capsize=6,mfc='white',label=case)
    axs[irow,icol].legend()
    # axs[irow,icol].set_ylim(ylim)

    icol=1
    def tFunc(dat):
        t=np.mean(dat,axis=0)
        t=t[0]-(-t[1]**2)
        t=np.real(t)
        tau=5
        tP=np.roll(t,-tau,axis=0)
        tM=np.roll(t,+tau,axis=0)
        t=np.arccosh( (tP+tM)/(2*t) )/tau
        return t
    for case in ['piC','pi0_conn','pi0_disc','pi0']:
        dat=getDat(case)
        (mean,cov,err)=yu.jackknife(dat,tFunc)
        
        tmin=2;tmax=45+1
        tMean=mean[tmin:tmax];tErr=err[tmin:tmax]

        colorDic={'piC':'r','pi0':'g','pi0_conn':'b','pi0_disc':'purple'}
        axs[irow,icol].errorbar(range(tmin,tmax),tMean,tErr,color=colorDic[case],fmt='s',elinewidth=2,capsize=6,mfc='white',label=case)
    axs[irow,icol].legend()
    axs[irow,icol].set_ylim(0,0.6)   

In [ ]:
class LatticeEnsemble:
    hbarc = 1/197.3 # hbarc = fm * MeV
    def __init__(self, ensemble):
        if ensemble == 'cA2.09.48':
            self.a=0.0938; self.L=4.50; self.ampi=0.06208; self.amN=0.4436
            self.info='cSW=1.57551, beta=2.1, Nf=2, V=48^3*96'
        elif ensemble == 'cA211.30.32':
            self.a=0.0947; self.L=3.03; self.ampi=0.12530; self.amN=0.5073
        elif ensemble == 'cA211.53.24':
            self.a=0.0947; self.L=2.27; self.ampi=0.16626; self.amN=0.56; self.ampi0=0.13
        else:
            print(ensemble+' not implemented')
        self.aInv=1/(self.a*self.hbarc); self.tpiL=(2*math.pi)/(self.L*self.hbarc); 
        self.mpiL=self.ampi * self.aInv * self.L * self.hbarc
        self.mpi=self.ampi*self.aInv; self.mN=self.amN*self.aInv
        self.mpi0=self.ampi0*self.aInv
    def getaEpi(self, n2):
        return np.sqrt(self.mpi**2+self.tpiL**2*n2)*self.a*self.hbarc
    def getaEpi0(self, n2):
        return np.sqrt(self.mpi0**2+self.tpiL**2*n2)*self.a*self.hbarc
    def getaEN(self, n2):
        return np.sqrt(self.mN**2+self.tpiL**2*n2)*self.a*self.hbarc
    def amass2mass(self, amass):
        return amass*self.aInv

lat2=LatticeEnsemble('cA211.53.24') 
print(lat2.getaEpi(1), lat2.getaEpi0(1))

# Previous

In [ ]:
i_mom=momDic[(0,0,0)]

def getDat(case):
    if case == 'piC':
        t=np.array([[(data['PPUP'][cfg,:,i_mom]+data['PPDN'][cfg,:,i_mom])/2,0] for cfg in range(len(cfgList))],dtype=object)
        return t
    elif case == 'pi0_conn':
        t=np.array([[(data['P0UP'][cfg,:,i_mom]+data['P0DN'][cfg,:,i_mom])/2,0] for cfg in range(len(cfgList))],dtype=object)
        return t
    elif case == 'pi0_disc':
        t=np.array([[data['pi0DoubleLoop'][cfg,:,i_mom],data['pi0Loop_g5_vev'][cfg]] for cfg in range(len(cfgList))],dtype=object)
        if i_mom!=momDic[(0,0,0)]:
            t=np.array([[data['pi0DoubleLoop'][cfg,:,i_mom],0] for cfg in range(len(cfgList))],dtype=object)
        return t
    elif case == 'pi0':
        t=np.array([[(data['P0UP'][cfg,:,i_mom]+data['P0DN'][cfg,:,i_mom])/2+data['pi0DoubleLoop'][cfg,:,i_mom],data['pi0Loop_g5_vev'][cfg]] for cfg in range(len(cfgList))],dtype=object)
        if i_mom!=momDic[(0,0,0)]:
            t=np.array([[(data['P0UP'][cfg,:,i_mom]+data['P0DN'][cfg,:,i_mom])/2+data['pi0DoubleLoop'][cfg,:,i_mom],0] for cfg in range(len(cfgList))],dtype=object)
        return t

tFunc=None

def run(ylim):
    (Nrow, Ncol) = (1, 1)
    fig, axs = plt.subplots(Nrow, Ncol, figsize=(5*Ncol*2, 4*Nrow*2), squeeze=False)
    irow=0;icol=0

    for case in ['piC','pi0_conn','pi0_disc','pi0']:
        dat=getDat(case)
        (mean,cov,err)=yu.jackknife(dat,tFunc)
        
        tmin=2;tmax=45+1
        tMean=mean[tmin:tmax];tErr=err[tmin:tmax]

        colorDic={'piC':'r','pi0':'g','pi0_conn':'b','pi0_disc':'purple'}
        axs[irow,icol].errorbar(range(tmin,tmax),tMean,tErr,color=colorDic[case],fmt='s',elinewidth=2,capsize=6,mfc='white',label=case)

    axs[irow,icol].legend()
    if ylim != None:
        axs[irow,icol].set_ylim(ylim)

def tFunc(dat):
    t=np.mean(dat,axis=0)
    t=t[0]-(-t[1]**2)
    t=np.real(t)
    return t
run(None)

# def tFunc(dat):
#     t=np.mean(dat,axis=0)
#     t=t[0]-(-t[1]**2)
#     t=np.real(t)
#     t=np.log(t/np.roll(t,-1,axis=0))
#     return t
# # run(None)
# run([-0.3,0.3])

# def tFunc(dat):
#     t=np.mean(dat,axis=0)
#     t=t[0]-(-t[1]**2)
#     t=np.real(t)
#     tau=2
#     tP=np.roll(t,-tau,axis=0)
#     tM=np.roll(t,+tau,axis=0)
#     t=np.abs(np.log((tM+np.sqrt(tM**2-t[t_total//2]**2))/(tP+np.sqrt(tP**2-t[t_total//2]**2))))/(2*tau)
#     return t
# # run([None])
# run([0,0.3])

def tFunc(dat):
    t=np.mean(dat,axis=0)
    t=t[0]-(-t[1]**2)
    t=np.real(t)
    tau=5
    tP=np.roll(t,-tau,axis=0)
    tM=np.roll(t,+tau,axis=0)
    t=np.arccosh( (tP+tM)/(2*t) )/tau
    return t
# run(None)
# run([0,0.3])
run([0,0.5])


In [ ]:
(Nrow, Ncol) = (1, 1)
fig, axs = plt.subplots(Nrow, Ncol, figsize=(5*Ncol*2, 4*Nrow*2), squeeze=False)
irow=0;icol=0

tau=5

def run(fitMin,fitMax):
    for case in ['piC','pi0_conn','pi0_disc','pi0']:
        # if case not in ['pi0_conn']:
        #     continue
        dat=getDat(case)

        def tFunc(dat):
            t=np.mean(dat,axis=0)
            t=t[0]-(-t[1]**2)
            t=np.real(t)
            t=(t+np.flip(np.roll(t,-1)))/2

            tP=np.roll(t,-tau,axis=0)
            tM=np.roll(t,+tau,axis=0)
            t=np.arccosh( (tP+tM)/(2*t) )/tau

            t=t[fitMin:fitMax]
            return t

        (mean,cov,err)=yu.jackknife(dat,tFunc)
        fitCov=cov
        
        def tFunc(dat):
            t=np.mean(dat,axis=0)
            t=t[0]-(-t[1]**2)
            t=np.real(t)
            t=(t+np.flip(np.roll(t,-1)))/2

            tP=np.roll(t,-tau,axis=0)
            tM=np.roll(t,+tau,axis=0)
            t=np.arccosh( (tP+tM)/(2*t) )/tau
        
            xList=np.arange(fitMin,fitMax)
            fitData=t[fitMin:fitMax]
            fitFunc=lambda tf,mN:mN
            paraInit=[0.15]
            fitMean=curve_fit(fitFunc,xList,fitData,paraInit,sigma=fitCov)[0]

            chi2Pre=fitFunc(xList,*fitMean)-fitData
            chi2=chi2Pre.dot(np.linalg.inv(fitCov)).dot(chi2Pre.T)
            dof=len(fitData)-len(paraInit)
            chi2Dof=chi2/dof

            return np.array([fitMean]+[np.array([chi2,dof,chi2Dof])],dtype=object)

        try:
            (mean,cov,err)=yu.jackknife(dat,tFunc,2)
        except:
            print("case="+case+" failed!")
            continue

        # print(mean)
        # print(err)

        tMean=mean[0]
        tErr=err[0]
        chi2R=mean[1][2]

        print("case="+case,yu.npRound([tMean[0],tErr[0]],5),\
              yu.npRound([lat.amass2mass(tMean[0]),lat.amass2mass(tErr[0])],5))

        colorDic={'piC':'r','pi0':'g','pi0_conn':'b','pi0_disc':'purple'}
        axs[irow,icol].errorbar([fitMin],tMean,tErr,color=colorDic[case],fmt='s',elinewidth=2,capsize=6,mfc='white',label=case)
        axs[irow,icol].annotate("%0.1f" %chi2R,(fitMin-0.2,tMean-tErr-0.005),color='black')

fitMax=24
fitMinMin=tau+1+1
for fitMin in range(fitMinMin,fitMax-1):
    # if fitMin != 11:
    #     continue
    print("fitMin="+str(fitMin))
    run(fitMin,fitMax)
    print()

    if fitMin==fitMinMin:
        axs[irow,icol].legend()
        # axs[irow,icol].set_ylim()


# (0,0,1)

In [ ]:
# inPath = '/p/project/pines/li47/code/projectData/nucleon_sigma_term/cA211.53.24/QuarkLoops_pi0_insertion_post/avg/'
# outPath = inPath

# with h5py.File(inPath+'pi0Loop-12.h5') as fr:
#     momList=fr[list(fr.keys())[0]]['mvec'][()]
#     momDic={}
#     for i in range(len(momList)):
#         momDic[tuple(momList[i])]=i
#     momListNega=-momList
#     momNegaIndexList=[momDic[tuple(momListNega[i])] for i in range(len(momList))]


# t_total=48
# mom=(0,0,0)

# def run(tFunc,ylim):
#     with h5py.File(outPath+'pi0DoubleLoop-12.h5') as f:
#         with h5py.File(outPath+'pi0Loop-12.h5') as fL:
#             data=np.array([[f[cfg]['pi0DoubleLoop'][:,momDic[mom]],fL[cfg+'/pi0Loop_g5_vev'][()]] for cfg in f.keys()])
    
#     dat=data

#     (mean,cov,err)=yu.jackknife(dat,tFunc)

#     (Nrow, Ncol) = (1, 1)
#     fig, axs = plt.subplots(Nrow, Ncol, figsize=(5*Ncol, 4*Nrow), squeeze=False)
#     irow=0;icol=0

#     tmin=1;tmax=47
#     tMean=mean[tmin:tmax];tErr=err[tmin:tmax]

#     axs[irow,icol].errorbar(range(tmin,tmax),tMean,tErr,color='b',fmt='s',elinewidth=2,capsize=6,mfc='white')

#     # print(mean)
#     # print(err)


#     with h5py.File(outPath+'pi0DoubleLoop.h5') as f:
#         with h5py.File(outPath+'pi0Loop.h5') as fL:
#             data=np.array([[f[cfg]['pi0DoubleLoop'][:,momDic[mom]],fL[cfg+'/pi0Loop_g5_vev'][()]] for cfg in f.keys()])
    
#     dat=data

#     (mean,cov,err)=yu.jackknife(dat,tFunc)
#     tMean=mean[tmin:tmax];tErr=err[tmin:tmax]

#     axs[irow,icol].errorbar(range(tmin,tmax),tMean,tErr,color='r',fmt='s',elinewidth=2,capsize=6,mfc='white')

#     axs[irow,icol].set_ylim(ylim)

# def tFunc(dat):
#     t=np.mean(dat,axis=0)
#     t=t[0]-(-t[1]**2)
#     t=np.real(t)
#     return t

# run(tFunc,[0,6])

# def tFunc(dat):
#     t=np.mean(dat,axis=0)
#     t=t[0]-(-t[1]**2)
#     t=np.real(t)
#     t=np.log(t/np.roll(t,-1,axis=0))
#     return t

# run(tFunc,[-0.15,0.15])

# def massFunc(mN):
#     tfList=np.arange(t_total)
#     return (np.exp(-mN*tfList)+np.exp(-mN*(t_total-tfList)))/(np.exp(-mN*(tfList+1))+np.exp(-mN*(t_total-(tfList+1))))

# def tFunc(dat):
#     t=np.mean(dat,axis=0)
#     t=t[0]-(-t[1]**2)
#     t=np.real(t)
#     t=t/np.roll(t,-1,axis=0)
#     tfList=np.arange(t_total)
#     def tempFunc(mN):
#         return (np.exp(-mN*tfList)+np.exp(-mN*(t_total-tfList)))/(np.exp(-mN*(tfList+1))+np.exp(-mN*(t_total-(tfList+1))))-t
#     t = fsolve(tempFunc,np.zeros(t_total)+0.1)
#     return t

# run(tFunc,[0,0.2])


In [ ]:
# inPath = '/p/project/pines/li47/code/projectData/nucleon_sigma_term/cA211.53.24/QuarkLoops_pi0_insertion_post/avg/'
# outPath = inPath

# with h5py.File(outPath+'pi0DoubleLoop.h5') as f:
#     with h5py.File(outPath+'pi0Loop.h5') as fL:
#         data=np.array([[f[cfg]['pi0DoubleLoop'][:,momDic[mom]],fL[cfg+'/pi0Loop_g5_vev'][()]] for cfg in f.keys()])


# dat=data

# def tFunc(dat):
#     t=np.mean(dat,axis=0)
#     t=t[0]-(-t[1]**2)
#     t=np.real(t)
#     t=t+np.flip(np.roll(t,-1))
#     # t=np.log(t/np.roll(t,-1,axis=0))
#     return t

# (mean,cov,err)=yu.jackknife(dat,tFunc)
# # print(mean)
# # print(cov)
 
# print()

# min=0;max=24

# print(np.linalg.eigvals(cov[min:max,min:max]))

In [ ]:
# with h5py.File(outPath+'pi0DoubleLoop-12.h5') as f:
#     with h5py.File(outPath+'pi0Loop-12.h5') as fL:
#         data=np.array([[f[cfg]['pi0DoubleLoop'][:,momDic[mom]],fL[cfg+'/pi0Loop_g5_vev'][()]] for cfg in f.keys()])

# dat=data

# def tFunc(dat):
#     t=np.mean(dat,axis=0)
#     t=t[0]-(-t[1]**2)
#     t=np.real(t)
#     return t

# (mean,cov,err)=yu.jackknife(dat,tFunc)

# fitMin=8;fitMax=48-fitMin
# fitCov=cov[fitMin:fitMax,fitMin:fitMax]
# # print(fitCov)

# def tFunc(dat):
#     t=np.mean(dat,axis=0)
#     t=t[0]-(-t[1]**2)
#     t=np.real(t)

#     xList=np.arange(fitMin,fitMax)
#     fitData=t[fitMin:fitMax]
#     fitFunc=lambda tf,mN,c0: c0*(np.exp(-mN*tf)+np.exp(-mN*(t_total-tf)))
#     paraInit=[0.1,1]
#     fitMean=curve_fit(fitFunc,xList,fitData,paraInit,sigma=fitCov)[0]

#     chi2Pre=fitFunc(xList,*fitMean)-fitData
#     chi2=chi2Pre.dot(np.linalg.inv(fitCov)).dot(chi2Pre.T)
#     dof=len(fitData)-len(paraInit)
#     chi2Dof=chi2/dof

#     return np.array([fitMean]+[np.array([chi2,dof,chi2Dof])],dtype=object)

# (mean,cov,err)=yu.jackknife(dat,tFunc,2)
# print(mean,err)


# with h5py.File(outPath+'pi0DoubleLoop.h5') as f:
#     with h5py.File(outPath+'pi0Loop.h5') as fL:
#         data=np.array([[f[cfg]['pi0DoubleLoop'][:,momDic[mom]],fL[cfg+'/pi0Loop_g5_vev'][()]] for cfg in f.keys()])

# dat=data

# def tFunc(dat):
#     t=np.mean(dat,axis=0)
#     t=t[0]-(-t[1]**2)
#     t=np.real(t)
#     return t

# (mean,cov,err)=yu.jackknife(dat,tFunc)

# fitCov=cov[fitMin:fitMax,fitMin:fitMax]

# def tFunc(dat):
#     t=np.mean(dat,axis=0)
#     t=t[0]-(-t[1]**2)
#     t=np.real(t)

#     xList=np.arange(fitMin,fitMax)
#     fitData=t[fitMin:fitMax]
#     fitFunc=lambda tf,mN,c0: c0*(np.exp(-mN*tf)+np.exp(-mN*(t_total-tf)))
#     paraInit=[0.1,1]
#     fitMean=curve_fit(fitFunc,xList,fitData,paraInit,sigma=fitCov)[0]

#     chi2Pre=fitFunc(xList,*fitMean)-fitData
#     chi2=chi2Pre.dot(np.linalg.inv(fitCov)).dot(chi2Pre.T)
#     dof=len(fitData)-len(paraInit)
#     chi2Dof=chi2/dof

#     return np.array([fitMean]+[np.array([chi2,dof,chi2Dof])],dtype=object)

# (mean,cov,err)=yu.jackknife(dat,tFunc,2)
# print(mean,err)

# print(lat.ampi)

In [ ]:
# inPath = '/p/project/pines/li47/code/scratch/run/nucleon_sigma_term/cA211.53.24/QuarkLoops_pi0_insertion/post_data/'
# outPath = inPath

# with h5py.File(inPath+'pi0Loop-12.h5') as fr:
#     momList=fr[list(fr.keys())[0]]['mvec'][()]
#     momDic={}
#     for i in range(len(momList)):
#         momDic[tuple(momList[i])]=i
#     momListNega=-momList
#     momNegaIndexList=[momDic[tuple(momListNega[i])] for i in range(len(momList))]


# t_total=48
# mom=(0,0,1)

# def run(tFunc,ylim):
#     with h5py.File(outPath+'pi0DoubleLoop-12.h5') as f:
#         data=np.array([f[cfg]['pi0DoubleLoop'] for cfg in f.keys()])

#     dat=data[:,:,momDic[mom]]

#     (mean,cov,err)=yu.jackknife(dat,tFunc)

#     (Nrow, Ncol) = (1, 1)
#     fig, axs = plt.subplots(Nrow, Ncol, figsize=(5*Ncol, 4*Nrow), squeeze=False)
#     irow=0;icol=0

#     tmin=1;tmax=47
#     tMean=mean[tmin:tmax];tErr=err[tmin:tmax]

#     axs[irow,icol].errorbar(range(tmin,tmax),tMean,tErr,color='b',fmt='s',elinewidth=2,capsize=6,mfc='white')

#     # print(mean)
#     # print(err)


#     with h5py.File(outPath+'pi0DoubleLoop.h5') as f:
#         data=np.array([f[cfg]['pi0DoubleLoop'] for cfg in f.keys()])

#     dat=data[:,:,momDic[mom]]

#     (mean,cov,err)=yu.jackknife(dat,tFunc)
#     tMean=mean[tmin:tmax];tErr=err[tmin:tmax]

#     axs[irow,icol].errorbar(range(tmin,tmax),tMean,tErr,color='r',fmt='s',elinewidth=2,capsize=6,mfc='white')

#     axs[irow,icol].set_ylim(ylim)

# def tFunc(dat):
#     t=np.mean(dat,axis=0)
#     t=np.real(t)
#     return t

# run(tFunc,[0,0.6])

# def tFunc(dat):
#     t=np.mean(dat,axis=0)
#     t=np.real(t)
#     t=np.log(t/np.roll(t,-1,axis=0))
#     return t

# run(tFunc,[-0.8,0.8])

# def massFunc(mN):
#     tfList=np.arange(t_total)
#     return (np.exp(-mN*tfList)+np.exp(-mN*(t_total-tfList)))/(np.exp(-mN*(tfList+1))+np.exp(-mN*(t_total-(tfList+1))))

# def tFunc(dat):
#     t=np.mean(dat,axis=0)
#     t=np.real(t)
#     t=t/np.roll(t,-1,axis=0)
#     tfList=np.arange(t_total)
#     def tempFunc(mN):
#         return (np.exp(-mN*tfList)+np.exp(-mN*(t_total-tfList)))/(np.exp(-mN*(tfList+1))+np.exp(-mN*(t_total-(tfList+1))))-t
#     t = fsolve(tempFunc,np.zeros(t_total)+0.1)
#     return t

# run(tFunc,[0,0.2])